In [1]:
from split_data import split

Imported


In [3]:
split("../Data/Original/Apple", "../training_data/", "../test_data/", 0.5)


---------------------
Finish splitting Apple___Cedar_apple_rust
[Training Samples:137, Testing Samples:138]   Original Samples:275
---------------------
Finish splitting Apple___Apple_scab
[Training Samples:315, Testing Samples:315]   Original Samples:630
---------------------
Finish splitting Apple___Black_rot
[Training Samples:311, Testing Samples:310]   Original Samples:621
---------------------
Finish splitting Apple___healthy
[Training Samples:823, Testing Samples:822]   Original Samples:1645
